In [7]:
# Block 0: Imports
import os, sys
import glob
import time
import yaml
import requests
import time
import zipfile
import shutil
import csv
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import random
import backoff
import pandas as pd
import numpy as np
import datetime
import json
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "Scripts")))

from clip_ALPOD_to_SR_extent import clip_vector_with_geometry, extract_geospatial_info_from_xml
from mask_clouds_and_classify_ice import calculate_output_rasters
from calculate_ice_cover_statistics_per_lake import calculate_lake_statistics

In [8]:
# Block 1: Config for lake classification

config = {
    #UDM mask bands to remove data beneath
    'udm_mask_bands': [3, 4, 5, 6], # 3 - Shadow, 4 - Light Haze, 5 - Heavy Haze (Depracated but kept in case any are UDM 2.0), 6 - Cloud
   
    #SR image bands to keep in the final TIF files
    'sr_keep_bands': [3], #3 - Red

    'pixel_reflectance_thresholds': {
        'Ice': (950, 3800),  # Could change to 1100-- silty problem persists
        'Snow': (3800, float('inf')),
        'Water': (float('-inf'), 950)
    }
}

study_sites_to_process = {
    'YF': [
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2019", 
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2020",
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2021", 
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2022",
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2023", 
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2024", 
        r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\Breakup_2025",
    ],
}

output_path = r"E:\planetscope_lake_ice\Data\Output"

alpod_vector_shapefiles = {
    'YF': r"E:\planetscope_lake_ice\Data\Input\YF 50x50 km\YF Lakes from ALPOD Shapefile\YF_50x50km_lakes.shp",
}

In [9]:
# Block 2: Image Processing Function 

def process_planetscope_image(sr_image_path, study_site, lake_vector_shapefile, site_output_path):
    """
    For the given PlanetScope SR TIFF:
      0) build output folders, and find its accompanying UDM & XML
      1) delete everything but sr_keep_band to reduce size of TIFs in deep storage, create cloud mask as uint 8 file, create classified mask as a uint8 file
      2) mask red band, threshold/classify ice & snow
      3) append lake stats to NetCDF
    """
    
    # ────────────────────────────────────────────────────────────────────────────────
    # 0: Create output folders // find correlated XML & UDM for this SR image
    # ────────────────────────────────────────────────────────────────────────────────
    # build image name strings for processing dates
    sr_filename = os.path.basename(sr_image_path) # E.g. 20200415_222212_87_1060_ortho_analytic_4b_sr.tif
    sr_filename_no_ext = os.path.splitext(sr_filename)[0] # E.g. 20200415_222212_87_1060_ortho_analytic_4b_sr
    image_core_name = sr_filename_no_ext.split('_ortho', 1)[0] ## E.g. 20200415_222212_87_1060

    # Get the full directory path where the SR image is located
    sr_image_directory = os.path.dirname(sr_image_path)
    
    # identify the current season folder (e.g. "Breakup_2019") and build output paths based on that season
    input_season_folder = os.path.basename(sr_image_directory) # e.g. "Breakup_2019 from downloads - read from global config"
    output_rasters_dir = os.path.join(site_output_path, "Rasters", input_season_folder) 
    output_shapefile_dir = os.path.join(site_output_path, "Shapefiles", input_season_folder, sr_filename_no_ext)
    for d in (output_rasters_dir, output_shapefile_dir): 
        os.makedirs(d, exist_ok=True)

    # locate UDM and XML metadata using glob to find matching files in the same directory
    udm_pattern = os.path.join(sr_image_directory, f"{image_core_name}*udm2.tif")
    xml_pattern = os.path.join(sr_image_directory, f"{image_core_name}*.xml")
    
    udm_files = glob.glob(udm_pattern)
    xml_files = glob.glob(xml_pattern)
    
    if not udm_files or not xml_files:
        raise FileNotFoundError(f"\n######################\nERROR: UDM or XML files not found for {sr_filename}\nLooked for UDM: {udm_pattern}\nLooked for XML: {xml_pattern}\n######################\n")
    
    udm_path = udm_files[0]  # Take the first match
    xml_path = xml_files[0]  # Take the first match

    print(f"All pre-processing complete. Beginning classification for image {sr_filename}.\n")

    # ────────────────────────────────────────────────────────────────────────────────
    # 1: Clip study site lake mask to the image footprint (sourced from the XML file) so only lakes
    #  which fall entirely within this SR image's extent are considered
    # ────────────────────────────────────────────────────────────────────────────────
    output_vector_path = os.path.join(output_shapefile_dir, f"{image_core_name}_lakes.shp") # The lakes which fall within the image's extent

    print("Clipping geometry...")
    geo_info = extract_geospatial_info_from_xml(xml_path)

    clip_vector_with_geometry(
        lake_vector_shapefile,
        geo_info['geometry'],
        output_vector_path
    )

    print("Geometry clipped successfully.\n")

    # ────────────────────────────────────────────────────────────────────────────────
    # 2: Delete everything but sr_keep_band to reduce size of TIFs in deep storage, create cloud mask as uint 8 file, create classified mask as a uint8 file, all with lzw compression
    # ────────────────────────────────────────────────────────────────────────────────

    output_sr_path = os.path.join(output_rasters_dir, "Single_Band_Rasters_Uint16", sr_filename)
    output_udm_path = os.path.join(output_rasters_dir, "Cloud_Masks_Uint8", f"{image_core_name}_cloud_mask.tif")
    output_classified_path = os.path.join(output_rasters_dir, "Ice_Snow_Water_Classified_Masks_Uint8", f"{image_core_name}_classified_ice_snow.tif")
    
    # Make sure the subdirectories exist
    for subdir_path in [os.path.dirname(output_sr_path), os.path.dirname(output_udm_path), os.path.dirname(output_classified_path)]:
        os.makedirs(subdir_path, exist_ok=True)
    
    print("Creating cloud mask, clipping rasters...")
    calculate_output_rasters(
        sr_image_path,
        udm_path,
        config['udm_mask_bands'],  # Fixed: was 'mask_bands', should be 'udm_mask_bands'
        config['sr_keep_bands'],   # Fixed: was 'keep_bands', should be 'sr_keep_bands'
        output_sr_path,
        output_udm_path,
        output_classified_path
    )

    print(f"     Clouds masked successfully.")

    # ────────────────────────────────────────────────────────────────────────────────
    # 3: Calculate statistics for each lake, and then add the statistics to an excel sheet
    # ────────────────────────────────────────────────────────────────────────────────

    print(f"Calculating lake statistics...")
    calculate_lake_statistics(
        output_classified_path,
        image_core_name,
        site_output_path,
        study_site,
        lake_vector_shapefile
    )

    print(f"Finished processing {image_core_name}\n# ──────────────────────────────────────────────────────────────────────────────── #\n")

In [10]:
# Block 3: Loop through all images to clip, clean, and classify lake ice cover

for study_site, paths_list in study_sites_to_process.items():
    lake_vector_shapefile = alpod_vector_shapefiles[study_site]  # Get corresponding shapefile

    for site_folder in paths_list:
        pattern = os.path.join(site_folder, "*.tif")  # Match all .tif files
        for sr_tif_path in glob.glob(pattern):
            if 'udm' not in os.path.basename(sr_tif_path).lower():  # Exclude files with 'udm' so only SR goes through
                try:
                    site_output_path = os.path.join(output_path, study_site)

                    # Then pass it to the function
                    process_planetscope_image(sr_tif_path, study_site, lake_vector_shapefile, site_output_path)

                except Exception as e:
                    print(f"Error processing {sr_tif_path}: {e}")

In [11]:
import os
import glob
import json
import datetime
import pandas as pd

def parse_histogram(histogram_str):
    try:
        histogram = json.loads(histogram_str)
        total_pixels = sum(histogram.values())
        invalid_pixels = histogram.get('0', 0) + histogram.get('255', 0)
        invalid_percent = (invalid_pixels / total_pixels) * 100 if total_pixels > 0 else 100

        ice_pixels = histogram.get('1', 0)
        snow_pixels = histogram.get('2', 0)
        water_pixels = histogram.get('3', 0)

        valid_pixels = ice_pixels + snow_pixels + water_pixels
        if valid_pixels == 0:
            return 0, 0, 0, invalid_percent, False

        return (
            (ice_pixels / valid_pixels) * 100,
            (snow_pixels / valid_pixels) * 100,
            (water_pixels / valid_pixels) * 100,
            invalid_percent,
            True
        )
    except (json.JSONDecodeError, ValueError, ZeroDivisionError):
        return 0, 0, 0, 100, False

def aggregate_daily_observations(df):
    df['date'] = df['datetime'].dt.date
    daily_data = []
    for date, group in df.groupby('date'):
        max_water_idx = group['water_percent'].idxmax()
        best_obs = group.loc[max_water_idx]
        datetime_repr = best_obs['datetime'].replace(hour=12, minute=0, second=0, microsecond=0)
        daily_data.append({
            'datetime': datetime_repr,
            'date': date,
            'ice_percent': best_obs['ice_percent'],
            'snow_percent': best_obs['snow_percent'],
            'water_percent': best_obs['water_percent']
        })
    return pd.DataFrame(daily_data).sort_values('datetime').reset_index(drop=True)

def classify_pixel_validity(df_daily):
    df_daily['pixel_valid'] = True

    for i, row in df_daily.iterrows():
        if row['ice_percent'] < 10 and row['snow_percent'] < 10 and row['water_percent'] < 10:
            df_daily.loc[i, 'pixel_valid'] = False

    for i, row in df_daily.iterrows():
        if sum([row['ice_percent'] == 100, row['snow_percent'] == 100, row['water_percent'] == 100]) > 1:
            df_daily.loc[i, 'pixel_valid'] = False

    if len(df_daily) >= 3:
        for i in range(1, len(df_daily) - 1):
            if not df_daily.loc[df_daily.index[i - 1], 'pixel_valid']:
                continue
            if not df_daily.loc[df_daily.index[i + 1], 'pixel_valid']:
                continue
            for class_name in ['ice_percent', 'snow_percent']:
                prev_val = df_daily.loc[df_daily.index[i - 1], class_name]
                curr_val = df_daily.loc[df_daily.index[i], class_name]
                next_val = df_daily.loc[df_daily.index[i + 1], class_name]
                if (curr_val - prev_val) > 75 and (curr_val - next_val) > 75:
                    if abs(prev_val - next_val) < abs(curr_val - prev_val):
                        df_daily.loc[df_daily.index[i], 'pixel_valid'] = False
                        break

    first_high_water_idx = None
    for i, row in df_daily.iterrows():
        if row['pixel_valid'] and row['water_percent'] >= 75:
            first_high_water_idx = row.name
            break

    if first_high_water_idx is not None:
        start_pos = df_daily.index.get_loc(first_high_water_idx)
        for i in range(start_pos + 1, len(df_daily)):
            if df_daily.loc[df_daily.index[i], 'water_percent'] < 95:
                df_daily.loc[df_daily.index[i], 'pixel_valid'] = False

    return df_daily

def find_breakup_date(df_sorted):
    df_daily = aggregate_daily_observations(df_sorted)
    df_classified = classify_pixel_validity(df_daily)
    valid_df = df_classified[df_classified['pixel_valid']].reset_index(drop=True)

    for i in range(len(valid_df) - 1):
        curr = valid_df.iloc[i]
        next_ = valid_df.iloc[i + 1]
        if curr['water_percent'] <= 75 < next_['water_percent']:
            if next_['water_percent'] == curr['water_percent']:
                continue
            t1 = curr['datetime']
            t2 = next_['datetime']
            frac = (75 - curr['water_percent']) / (next_['water_percent'] - curr['water_percent'])
            breakup_date = t1 + (t2 - t1) * frac
            lower_bound_days = round((breakup_date - t1).total_seconds() / 86400)
            upper_bound_days = round((t2 - breakup_date).total_seconds() / 86400)
            return breakup_date, lower_bound_days, upper_bound_days, "WATER interpolated @75%"

    return None, None, None, "NO CROSSING FOUND"

def calculate_breakup_stats_continuous_multiyear(csv_folder_path, output_csv_path, cloud_threshold=33):
    csv_files = glob.glob(os.path.join(csv_folder_path, "*_ice_snow.csv"))
    results = []

    print(f"Found {len(csv_files)} lake CSVs...")

    for file_index, csv_file in enumerate(csv_files):
        lake_id = int(os.path.basename(csv_file).split('_')[0])
        try:
            df = pd.read_csv(csv_file)
            if df.empty:
                continue

            valid_data = []
            for _, r in df.iterrows():
                ice_pct, snow_pct, water_pct, invalid_pct, is_valid_hist = parse_histogram(r['histogram'])
                if is_valid_hist and invalid_pct <= cloud_threshold:
                    timestamp = datetime.datetime.fromtimestamp(r['unix_time'])
                    valid_data.append({
                        'datetime': timestamp,
                        'ice_percent': ice_pct,
                        'snow_percent': snow_pct,
                        'water_percent': water_pct
                    })

            if not valid_data:
                results.append({
                    'lake_id': lake_id,
                    'year': None,
                    'breakup_date': None,
                    'error_days_negative': None,
                    'error_days_positive': None,
                    'total_observations': len(df),
                    'valid_observations': 0,
                    'cloud_threshold_used': cloud_threshold,
                    'detection_method': f'ALL IMAGES > {cloud_threshold}% INVALID'
                })
                pd.DataFrame(results).to_csv(output_csv_path, index=False)
                continue

            df_valid = pd.DataFrame(valid_data)
            df_valid['year'] = df_valid['datetime'].dt.year
            df_valid['month'] = df_valid['datetime'].dt.month

            for year, year_group in df_valid.groupby('year'):
                # Only use spring months April–June
                spring_data = year_group[year_group['month'].between(4, 6)].sort_values('datetime')
                if spring_data.empty:
                    detection_method = 'NO SPRING DATA'
                    results.append({
                        'lake_id': lake_id,
                        'year': year,
                        'breakup_date': None,
                        'error_days_negative': None,
                        'error_days_positive': None,
                        'total_observations': len(year_group),
                        'valid_observations': 0,
                        'cloud_threshold_used': cloud_threshold,
                        'detection_method': detection_method
                    })
                    continue

                df_daily = aggregate_daily_observations(spring_data)
                if df_daily.empty:
                    detection_method = 'NO VALID DAILY DATA'
                    results.append({
                        'lake_id': lake_id,
                        'year': year,
                        'breakup_date': None,
                        'error_days_negative': None,
                        'error_days_positive': None,
                        'total_observations': len(spring_data),
                        'valid_observations': 0,
                        'cloud_threshold_used': cloud_threshold,
                        'detection_method': detection_method
                    })
                    continue

                first_row = df_daily.iloc[0]
                if not (first_row['ice_percent'] > first_row['water_percent'] or first_row['snow_percent'] > first_row['water_percent']):
                    detection_method = 'INVALID TIME SERIES - NO ICE START'
                    results.append({
                        'lake_id': lake_id,
                        'year': year,
                        'breakup_date': None,
                        'error_days_negative': None,
                        'error_days_positive': None,
                        'total_observations': len(spring_data),
                        'valid_observations': len(df_daily),
                        'cloud_threshold_used': cloud_threshold,
                        'detection_method': detection_method
                    })
                    continue

                breakup_date, before_days, after_days, detection_method = find_breakup_date(spring_data)
                results.append({
                    'lake_id': lake_id,
                    'year': year,
                    'breakup_date': breakup_date.strftime('%m/%d/%Y %I:%M:%S %p') if breakup_date else None,
                    'error_days_negative': before_days,
                    'error_days_positive': after_days,
                    'total_observations': len(spring_data),
                    'valid_observations': len(df_daily),
                    'cloud_threshold_used': cloud_threshold,
                    'detection_method': detection_method
                })

        except Exception as e:
            print(f"Error with {lake_id}: {e}")

        # Incremental write after every lake
        pd.DataFrame(results).to_csv(output_csv_path, index=False)

        if (file_index + 1) % 50 == 0:
            print(f"Processed {file_index + 1}/{len(csv_files)} lakes")

    print(f"Final CSV saved to {output_csv_path}")

# Example run
for study_site in ['YF']:
    csv_folder = os.path.join(output_path, study_site, "Lake Time Series CSVs")
    output_csv = os.path.join(output_path, study_site, f"lake_breakup_results_{study_site}.csv")
    calculate_breakup_stats_continuous_multiyear(csv_folder, output_csv, cloud_threshold=33)


Found 0 lake CSVs...
Final CSV saved to E:\planetscope_lake_ice\Data\Output\YF\lake_breakup_results_YF.csv
